In [1]:
# imports

from itertools import chain
import pylab as pb
import GPy as gpy
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline  
%config InlineBackend.figure_format = 'svg'
matplotlib.rcParams['figure.figsize'] = (8,5)

import numpy as np
from numpy import random as nprnd
from scipy import linalg as sciLA
from scipy import optimize

# Specify covariance constraints for a tensor of dimensionality 6

In [2]:
# specify tensor
nmodes = 6
sizes = nprnd.randint(10, size = nmodes)+1
# specify covariance constraints
# genarate marginal covariance matrices of size specified in dim
covConst = []
eigenValuesTruelist = []
eigenVectorsTruelist = []
scale = nprnd.randint(10)+1
for i in range(nmodes):
    eigVectors = sciLA.orth(nprnd.randn(sizes[i], sizes[i]))
    eigValues = abs(nprnd.randn(sizes[i]))
    eigValues = scale*(eigValues/np.sum(eigValues))
    Sigma = np.dot(np.dot(eigVectors, np.diag(eigValues)), eigVectors.T)
    covConst.append(Sigma)
    eigenValuesTruelist.append(eigValues)
    eigenVectorsTruelist.append(eigVectors)

In [3]:

def kron_mvprod(As, b):
    x = b
    numDraws = b[0, :].size
    CTN = b[:, 0].size
    for d in range(len(As)):
        A = As[d]
        Gd = A[:,0].size
        X = np.reshape(x, tuple([Gd, CTN*numDraws/Gd]), order = 'F')
        Z = np.dot(A, X).T
        x = np.reshape(Z, tuple([CTN,numDraws]), order = 'F')
    x = np.reshape(x, tuple([CTN*numDraws, 1]), order = 'F')
    x = np.reshape(x, tuple([numDraws, CTN]), order = 'F').T
    return x

In [4]:
def diagKronSum(Ds):
    nmodes = len(Ds)
    kronSumDs = np.zeros(1)
    for i in range(nmodes):
        kronSumDs = np.add(np.outer(kronSumDs, np.ones(len(Ds[i]))),np.outer(np.ones(len(kronSumDs)),Ds[i]))
        kronSumDs = np.hstack(kronSumDs.T)
    return kronSumDs
        

In [5]:
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#% <tensorMaxEntropy>
#% Copyright (C) 2016 Gamaleldin F. Elsayed and John P. Cunningham 
#%       (see full notice in README)
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#% sumA = sumTensor(A, sumDim)
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#% This function evaluates the sum of tensor at specific dimensions
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#% Inputs:
#%       - A: is the input n-dimesnional tensor
#%       - sumDim: the dimensions to sum over.
#% Outputs:
#%       - sumA: an n-dimensional tensor of the sum of tensor A at the 
#%       specified dimensions. The dimensions specified by sumDim will be of
#%       size 1.
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

def sumTensor(T, sumDim):
    sumT = T
    for i in range(len(sumDim)):
        sumT = np.expand_dims(np.sum(sumT, sumDim[i]), axis = sumDim[i])
    return sumT

In [16]:
def logObjectiveMaxEntropyTensor(logL, params):
    Lagrangians = []
    for x in params.tensorIxs:
        Lagrangians.append(np.exp(logL[np.add(sum(params.sizes[0:x]),range(int(params.sizes[x])))]))
############ building blocks of the gradient ###########
    LsTensor = diagKronSum(Lagrangians)
    LsTensor = np.reshape(LsTensor, tuple(params.sizes), order='F')                                 # kronecker sum of the lagrangian matrices eigenvalues
    invLsTensor = 1.0/LsTensor                                            # elementwise inverse of the above
    invSquareLsTensor = 1.0/LsTensor**2                                     # elementwise inverse square of the above
    Er = []
    logSums = []
    fx = []                                             # the log transformed cost decomposed to different tensor dimensions
    gradf_logL = np.zeros(len(logL))
    for x in params.tensorIxs:
        nxSet = list(set(params.tensorIxs).difference(set([x])))
        logSums.append(np.log(sumTensor(invLsTensor, nxSet)))                    # elementwise log of invLsTensor)
        Er.append(np.reshape(params.logEigValues[x], logSums[x].shape, order='F')- logSums[x])     # error with respect to each marginal covariance eigenvalue


        fx.append(np.reshape(Er[x], tuple([sizes[x], 1]), order='F')**2)                               
    f = sum(np.vstack(fx))/params.normalizeTerm


############ build the gradient from the blocks ###########
    for x in params.tensorIxs:
        nxSet = list(set(params.tensorIxs).difference(set([x])))
        gradfx_logLx = np.reshape(((2*Er[x]/sumTensor(invLsTensor, nxSet))*sumTensor(invSquareLsTensor, nxSet)),tuple([params.sizes[x],1]), order='F')*np.reshape(Lagrangians[x], tuple([params.sizes[x],1]), order='F')
        gradf_logLx = gradfx_logLx
        for y in nxSet:
            nySet = list(set(params.tensorIxs).difference(set([y])))
            nxySet = list(set(params.tensorIxs).difference(set([x, y])))
            gradfy_logLx = np.reshape(sumTensor((2*Er[y]/sumTensor(invLsTensor, nySet))*sumTensor(invSquareLsTensor, nxySet), [y]), tuple([params.sizes[x] ,1]), order='F')*np.reshape(Lagrangians[x], tuple([params.sizes[x] ,1]), order='F')
            gradf_logLx = gradf_logLx+gradfy_logLx

        gradf_logL[np.add(sum(params.sizes[0:x]),range(int(params.sizes[x])))] = gradf_logLx
    
    gradf_logL = gradf_logL/params.normalizeTerm  
############ return ###########
    return f, gradf_logL

        
        
        

In [17]:
# Gamaleldin Elsayed

# translated to python based on Carl Edward Rasmussen minimize.m function
# X is intial starting point
# obj_Fn is the function that takes (X, params) as inputs and spits back the current value of the objective and the gradient.
# length is the maximum allowed number of iterations.
# params any variables other than X for the obj_Fn to compute the cost value and the gradient.

def minimize(X, obj_Fn, length, params):
    #% imports:
    import numpy as np;
    
    supressOutPut = True;
    INT = 0.1;    # don't reevaluate within 0.1 of the limit of the current bracket
    EXT = 3.0;    # extrapolate maximum 3 times the current step-size
    MAX = 20;     # max 20 function evaluations per line search
    RATIO = 10.0;   # maximum allowed slope ratio
    SIG = 0.1; 
    RHO = SIG/2.0; # SIG and RHO are the constants controlling the Wolfe-
    # Powell conditions. SIG is the maximum allowed absolute ratio between
    # previous and new slopes (derivatives in the search direction), thus setting
    # SIG to low (positive) values forces higher precision in the line-searches.
    # RHO is the minimum allowed fraction of the expected (from the slope at the
    # initial point in the linesearch). Constants must satisfy 0 < RHO < SIG < 1.
    # Tuning of SIG (depending on the nature of the function to be optimized) may
    # speed up the minimization; it is probably not worth playing much with RHO.
    realmin = np.finfo(np.double).tiny 
    
    def unwrap(s):
        l = s.shape
        v = np.reshape(s, np.prod(l), order = 'F')
        return v
        
    def rewrap(s, v):
        l = s.shape;
        v = np.reshape(v, l, order = 'F')
        return v
        

    S='Linesearch'


    i = 0                                            # zero the run length counter
    ls_failed = False                             # no previous line search has failed
    f0, df0 = obj_Fn(X, params)    # get function value and gradient
    Z = X;X = unwrap(X); df0 = unwrap(df0);
    if not supressOutPut:
        print "%s %6i;  Value %4.6e\r" %(S, i, f0)
    fX = []
    fX.append(f0);

    s = -df0; 
    d0 = -np.dot(s.T,s)           # initial search direction (steepest) and slope
    x3 = 1.0/(1.0-d0)

    while i < length:              # while not finished
        i = i + 1                   # count iterations
        X0 = X; F0 = f0; dF0 = df0; # make a copy of current values
        M = MAX; 


        while True:   # keep extrapolating as long as necessary
            x2 = 0; f2 = f0; d2 = d0; f3 = f0; df3 = df0;
            success = False;

            while (not success) and (M > 0):
                try:
                    M = M - 1.0; i = i + 1;          # count epochs
                    f3, df3 = obj_Fn(rewrap(Z, X+x3*s), params);
                    df3 = unwrap(df3);
                    if np.isnan(f3) or np.isinf(f3) or any(np.isnan(df3)+np.isinf(df3)):
                        print "error(' ')"
                    success = True;
                except:         # catch any error which occured in f
                    x3 = (x2+x3)/2;  # bisect and try again


            if f3 < F0:
                X0 = X+x3*s; F0 = f3; dF0 = df3;       # keep best values
            d3 = np.dot(df3.T,s);                     # new slope


            if (d3 > (SIG*d0)) or (f3 > (f0+x3*RHO*d0)) or (M == 0):  # are we done extrapolating?
                break;


            x1 = x2; f1 = f2; d1 = d2;                      # move point 2 to point 1
            x2 = x3; f2 = f3; d2 = d3;                      # move point 3 to point 2
            A = 6*(f1-f2)+3*(d2+d1)*(x2-x1);                # make cubic extrapolation
            B = 3*(f2-f1)-(2*d1+d2)*(x2-x1);

            x3 = x1-d1*(x2-x1)**2/(B+np.sqrt(B*B-A*d1*(x2-x1))); # num. error possible, ok!
            if (not np.isreal(x3)) or np.isnan(x3) or np.isinf(x3) or (x3 < 0.): # num prob | wrong sign?
                x3 = x2*EXT;                    # extrapolate maximum amount
            elif x3 > x2*EXT:                  # new point beyond extrapolation limit?
                x3 = x2*EXT;                    # extrapolate maximum amount
            elif x3 < (x2+INT*(x2-x1)):         # new point too close to previous point?
                x3 = x2+INT*(x2-x1);
                                   ### end extrapolation ####

        while (abs(d3) > -SIG*d0 or (f3 > (f0+x3*RHO*d0))) and (M > 0):  # keep interpolating
            if d3 > 0 or f3 > f0+x3*RHO*d0:             # choose subinterval
                x4 = x3; f4 = f3; d4 = d3;               # move point 3 to point 4
            else:
                x2 = x3; f2 = f3; d2 = d3;               # move point 3 to point 2

            if f4 > f0:          
                x3 = x2-(0.5*d2*(x4-x2)**2)/(f4-f2-d2*(x4-x2)); # quadratic interpolation
            else:
                A = 6*(f2-f4)/(x4-x2)+3*(d4+d2);          # cubic interpolation
                B = 3*(f4-f2)-(2*d2+d4)*(x4-x2);
                x3 = x2+(np.sqrt(B*B-A*d2*(x4-x2)**2)-B)/A;   # num. error possible, ok!

            if np.isnan(x3) or np.isinf(x3):
                x3 = (x2+x4)/2;       # if we had a numerical problem then bisect


            x3 = max(min(x3, x4-INT*(x4-x2)),x2+INT*(x4-x2));  # don't accept too close
            f3, df3 = obj_Fn(rewrap(Z, X+x3*s), params);
            df3 = unwrap(df3);

            if f3 < F0:
                X0 = X+x3*s; F0 = f3; dF0 = df3;    # keep best values
            M = M - 1.0; i = i + 1;               # count epochs
            d3 = np.dot(df3.T,s);               # new slope
                               #### end interpolation ####

        if (abs(d3) < (-SIG*d0)) and (f3 < (f0+x3*RHO*d0)): # if line search succeeded
            X = X+x3*s; f0 = f3; fX.append(f0);              # update variables
            if not supressOutPut:
                print "%s %6i;  Value %4.6e\r" %(S, i, f0);
            s = np.dot((np.dot(df3.T, df3)-np.dot(df0.T,df3))/(np.dot(df0.T,df0)), s) - df3;  # Polack-Ribiere CG direction
            df0 = df3;                        # swap derivatives
            d3 = d0; d0 = np.dot(df0.T,s);
            if d0 > 0:                         # new slope must be negative
                s = -df0; d0 = -np.dot(s.T,s); # otherwise use steepest direction

            x3 = x3 * min(RATIO, d3/(d0-realmin));  # slope ratio but max RATIO
            ls_failed = False;                      # this line search did not fail
        else:
            X = X0; f0 = F0; df0 = dF0;            # restore best point so far
            if ls_failed or (i > abs(length)):      # line search failed twice in a row
                break;                             # or we ran out of time, so we give up

            s = -df0; d0 = -np.dot(s.T,s);           # try steepest
            x3 = 1.0/(1.0-d0);                     
            ls_failed = True;                        # this line search failed
    X = rewrap(Z,X);
    return X, fX, i
    

In [18]:
def optMaxEntropy(eigValues, maxIter, figFlg):
#  if the marginal covariances are low rank then the number of variables 
#  that we solve for are less. If full rank the number of variables that we 
#  solve for are equal to the sum of the tensor dimensions.
############# define optimization params class #####################
    class optParams:
        logEigValues = []
        nmodes = []
        tensorIxs = []
        sizes = []
        normalizeTerm = []
        logL = []
########################## initializations #########################
    params = optParams()
#     figFlg = True          # display summary figure flag
    params.nmodes = len(eigValues);              # tensor size; i.e. the number of different dimensions of the tensor
    sizes = np.zeros(params.nmodes)              # tensor dimensions
    params.tensorIxs = range(params.nmodes)  
    threshold = -10                                     # if an eigenvalue is below this threshold it is considered 0. 
    for x in params.tensorIxs:
        sizes[x] = len(eigValues[x])
    
# instead of solving for the largrangians directly we optimize latent variables that is equal to the log of the lagrangians

    preScale = sum(eigValues[0])/np.mean(sizes)         # prescale the eigenvalues for numerical stability
    params.logEigValues = []                            # the log of the eigenvalues
    params.sizes =  []                                  # true number of variables that we solve for, which is equal to the sum of the ranks of the marginal covariances                                                
    for x in params.tensorIxs:
        params.logEigValues.append(np.array([i for i in np.log(eigValues[x]/preScale) if i>threshold])) # eigenvalues should be order apriori
        params.sizes.append(len(params.logEigValues[x]))
    params.sizes = np.array(params.sizes)
    params.normalizeTerm = sum(np.hstack(params.logEigValues)**2)

####################### optimization step #############################
# initialization of the optimization variables    
    logL0 = []  
    for x in params.tensorIxs:
        nxSet = map(int, set(params.tensorIxs).difference(set([x])))
        logL0.append(np.log(sum(params.sizes[nxSet]))-params.logEigValues[x])
    
#     maxIter = 1000;        # maximum allowed iterations
    logL0 = np.array(np.hstack(logL0)).T
    logL, f, i = minimize(logL0 ,logObjectiveMaxEntropyTensor , maxIter, params); # this function performs all the optimzation heavy lifting
    L = np.exp(logL)
    params.L = L
    
    if figFlg:
        import matplotlib.pyplot as plt
        plt.plot(range(len(f)), f)
        plt.ylabel('objective fn')
        plt.show()
##################### convert solution from the lagrangian variables to the optimal eigenvalues of the big covariance matrix
    Lagrangians = []                                      # save the lagrangians to the output 
    for x in params.tensorIxs: 
        Lagrangians.append(np.hstack([L[np.add(sum(params.sizes[0:x]),range(int(params.sizes[x]))).astype(int)]/preScale, float('inf')*np.ones(sizes[x]-params.sizes[x])]))
    
    return Lagrangians, f[-1]
  

In [20]:
t = randTensors(sizes)
t.fitMaxEntropy(covConst)
ts = t.sampleTensors(10)

 /Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:92: RuntimeWarning:invalid value encountered in sqrt


Error in estimated marginal covariance of mode 0, empirically estimated from samples, is 0.21 %
Error in estimated marginal covariance of mode 1, empirically estimated from samples, is 0.67 %
Error in estimated marginal covariance of mode 2, empirically estimated from samples, is 0.89 %
Error in estimated marginal covariance of mode 3, empirically estimated from samples, is 0.51 %
Error in estimated marginal covariance of mode 4, empirically estimated from samples, is 0.93 %
Error in estimated marginal covariance of mode 5, empirically estimated from samples, is 0.27 %


In [ ]:
print ts[0].shape
t.samplingError(ts)

In [19]:
class randTensors:
    def __init__(self, sizes):
        self.sizes = sizes;
        self.marginalCov = [];
        self.nmodes = len(sizes);
        self.margCovs = [];                            # marginal Covariances of the distribution
        self.margEigVectors = [];                        # eigenvectors of the marginal Covariances of the distribution
        self.margEigValues = [];
        for i in range(self.nmodes):
            self.margCovs += [np.eye(sizes[i])/sizes[i]];
            self.margEigVectors += [np.eye(sizes[i])];
            self.margEigValues += [np.eye(sizes[i])];
        self.vecEigValues = np.ones(np.prod(self.sizes)); # eigenvectors of the vectorized distribution
    
    ###### function to check if a matrix is proper covariance matrix (i.e., symmetric posistive semidef)
    def isproperCov(self, Sigma):  ### gamal
        isCov = True
        SigmaSPD= (Sigma+Sigma.T)/2  # make sure it is symmetric pos def. matrix (i.e., proper covariance matrix)
        if not isCov:
            print ('The covariance matrix is not correct, approximating to the nearest possible covariance')
        return SigmaSPD;
    
    ###### function to compute the eigenvalues and eigenvectors of the marginal covariances
    def margEig(self, margCovs):
        margEigValues = [];
        margEigVectors = [];
        for i in range(len(margCovs)):
            Sigma_i = margCovs[i];
            Sigma_i = self.isproperCov(Sigma_i);
            Q, s, _ = sciLA.svd(Sigma_i)
            ix = np.argsort(s)[::-1]
            s = s[ix]
            Q = Q[:, ix]
            margEigValues += [s];
            margEigVectors += [Q];
        return margEigValues, margEigVectors
    
    def fitMaxEntropy(self, margCovs):
        margEigValues, margEigVectors = self.margEig(margCovs);
        maxIter = 1000;
        figFlg = False;
        Lagrangians, f = optMaxEntropy(margEigValues, maxIter, figFlg);
        if f>1e-10:
            print ("algorithm did not completely converged. error = %.10f \n results may be inaccurate" %f)
        self.vecEigValues = 1/diagKronSum(Lagrangians);
        self.margCovs = margCovs;
        self.margEigValues = margEigValues;
        self.margEigVectors = margEigVectors;
        
    def sampleTensors(self, nsamples):
        vecTensors = np.zeros([np.prod(self.sizes), nsamples]);
        for i in range(nsamples):
            vecTensors[:, i] = nprnd.randn(np.prod(self.sizes))
            vecTensors[:, i] = vecTensors[:, i]*np.sqrt(self.vecEigValues)
        vecTensors = np.real(kron_mvprod(self.margEigVectors, vecTensors))
        tensors = []
        for i in range(nsamples):
            tensors += [np.reshape(vecTensors[:,i], tuple(self.sizes), order = 'F')]
        
        _ = self.samplingError(tensors)
        return tensors
    
    def empiricalMargCovs(self, tensors):
        nsamples = len(tensors)
        M =  self.empiricalMean(tensors)
        sizes = np.array(tensors[0].shape)
        nmodes = len(sizes)
        allDim = range(nmodes)
        estMargCov = []
        for i in allDim:
            estMargCov.append(np.zeros([sizes[i], sizes[i]]))

        for j in range(nsamples):
            tensor = tensors[j]-M
            marginalCovs = []
            for i in allDim:
                niSet = list(set(allDim) - set([i]))
                z = np.reshape(np.transpose(tensor, list(chain.from_iterable([[i], niSet]))), tuple([sizes[i], np.prod(sizes[niSet])]), order = 'F')
                estMargCov[i] = (estMargCov[i]+(np.dot(z,z.T)/(nsamples-1)))
        for i in allDim:
            estMargCov[i] = (estMargCov[i])
        return estMargCov
    
    def empiricalMean(self, tensors):
        nsamples = len(tensors);
        M = tensors[0]/nsamples
        for i in range(nsamples-1):
            M = M+tensors[i+1]/nsamples
        return M
    
    def samplingError(self, tensors):
        nsamples = len(tensors);
        estMargCov = self.empiricalMargCovs(tensors);
        error = [];
        for i in range(self.nmodes):
            error += [(sciLA.norm(estMargCov[i]-self.margCovs[i], 'fro')/sciLA.norm(self.margCovs[i], 'fro'))**2*100];
            print "Error in estimated marginal covariance of mode %d, empirically estimated from samples, is %.2f %%" \
            %(i, error[i])


In [ ]:
def tensorNormal(covConst, numSamples):
    nmodes = len(covConst)
    sizes = []
    eigenValueslist = []
    eigenVectorslist = []
    for i in range(nmodes):
        Sigma_i = covConst[i]
        Sigma_i = (Sigma_i+Sigma_i.T)/2
        Q, s, _ = sciLA.svd(Sigma_i)
        ix = np.argsort(s)[::-1]
        s = s[ix]
        Q = Q[:, ix]
        sizes.append(len(s))
        eigenValueslist.append(s)
        eigenVectorslist.append(Q)
    
    sumEigValues = sum(eigenValueslist[0])

    scaleList = []
    for i in range(nmodes-1):
        scaleList.append(nprnd.rand()*sumEigValues**(1/nmodes))
    scaleList.append(sumEigValues/np.prod(scaleList))
    
    normalEigVectorslist = []
    normalSqrtEigValuesList = []
    for i in range(nmodes):
        W = scaleList[i]/sumEigValues*covConst[i];
        Q, s, _ = sciLA.svd(W)
        ix = np.argsort(s)[::-1]
        s = s[ix]
        Q = Q[:, ix]
        normalSqrtEigValuesList.append(np.diag(np.sqrt(s)))
        normalEigVectorslist.append(Q)

    vecTensors = nprnd.randn(np.prod(sizes), numSamples)
    vecTensors = np.real(kron_mvprod(normalSqrtEigValuesList, vecTensors))
    vecTensors = np.real(kron_mvprod(normalEigVectorslist, vecTensors))
    
    tensorList = []
    for i in range(numSamples):
        tensorList.append(np.reshape(vecTensors[:,i], tuple(sizes), order = 'F'))
        
    return tensorList

In [ ]:
numSamples = 100
tensorList = tensorNormal(covConst, numSamples)

estmarginalCovs = estTensorsMarginalCov(tensorList)
for i in range(len(covConst)):
    print "Error in covariance %d is %.2f %%" \
    %(i,(sciLA.norm(estmarginalCovs[i]-covConst[i], 'fro')/sciLA.norm(covConst[i], 'fro'))**2*100)